# Validación de datos SIVIGILA

## Carga de los datos

Este jupyter notebook esta generando los datos de un año en especifico (2012 el parquet part.0.parquet de SIVIGILA) para coincidir con las tablas compartidas del  power bi (Tablas agregadas) 

In [18]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("ValidaciónTablasSIVIGILA").getOrCreate()

In [19]:
#spark.stop()

In [20]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/sivigila/sivigila_parquet_2022/part.0.parquet")

df=df1_parquet

In [21]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 34152
Columnas del DataFrame unido: ['PersonaBasicaID', 'Evento', 'AñoEpidemiologico', 'SemanaEpidemilogica', 'Prestador', 'CodigoAdministradora', 'Administradora', 'Edad', 'UnidadMedidaEdad', 'Sexo', 'TipoRegimen', 'codigoOcupacion', 'Ocupacion', 'Etnia', 'ClasificacioninicialCaso', 'PacienteHospitalizado', 'SeguimientoFinalCaso', 'FechaNotificacion', 'FechaConsulta', 'FechaInicioSINTOmas', 'FechaHospitalizacion', 'FechaDefuncion', 'FechaNcto', 'FechaAjuste', 'GP_Discapacidad', 'GP_Despazados', 'GP_Migrantes', 'GP_Carcelarios', 'GP_Gestantes', 'GP_Indigenas', 'GP_PoblacionICBF', 'GP_MadresComunitarias', 'GP_Desmovilizados', 'GP_Psiquia', 'GP_VictimasViolencia', 'GP_otros', 'codigoPaisOcurrencia', 'PaisOcurrencia', 'codigoDepartamentoOcurrencia', 'DepartamentoOcurrencia', 'codigoMunicipioOcurrencia', 'MunicipioOcurrencia', 'codigoDepartamentoNotificacion', 'DepartamentoNotificacion', 'codigoMunicipioNotificacion', 'MunicipioNotificacion', 'codi

In [22]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 (128.343)
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 34152


In [23]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- PersonaBasicaID: string (nullable = true)
 |-- Evento: string (nullable = true)
 |-- AñoEpidemiologico: string (nullable = true)
 |-- SemanaEpidemilogica: string (nullable = true)
 |-- Prestador: string (nullable = true)
 |-- CodigoAdministradora: string (nullable = true)
 |-- Administradora: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- UnidadMedidaEdad: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- TipoRegimen: string (nullable = true)
 |-- codigoOcupacion: string (nullable = true)
 |-- Ocupacion: string (nullable = true)
 |-- Etnia: string (nullable = true)
 |-- ClasificacioninicialCaso: string (nullable = true)
 |-- PacienteHospitalizado: string (nullable = true)
 |-- SeguimientoFinalCaso: string (nullable = true)
 |-- FechaNotificacion: string (nullable = true)
 |-- FechaConsulta: string (nullable = true)
 |-- FechaInicioSINTOmas: string (nullable = true)
 |-- FechaHospitalizacion: string (nullable = true)
 |-- FechaDefuncion: stri

# Empieza el proceso de generación de los datos agrupados por atenciones

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'PersonaBasicaID','FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion'
    
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaBasicaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaBasicaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaBasicaID').agg(
    F.first('FechaInicioSINTOmas').alias('FechaInicioSINTOmas'),
    F.first('Edad').alias('Edad'),
    F.first('Sexo').alias('Sexo'),
    F.first('TipoRegimen').alias('TipoRegimen'),
    F.first('Evento').alias('Evento'),
    F.first('Area').alias('Area'),
    F.first('PacienteHospitalizado').alias('PacienteHospitalizado'),
    F.first('CondicionFinal').alias('CondicionFinal'),
    F.first('DiagnosticoDefuncion').alias('DiagnosticoDefuncion'), 
   
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------------+-------------------+----------+---------+----------------+--------------------+--------+---------------------+------------------+--------------------+---------------+
|PersonaBasicaID|FechaInicioSINTOmas|      Edad|     Sexo|     TipoRegimen|              Evento|    Area|PacienteHospitalizado|    CondicionFinal|DiagnosticoDefuncion|personaid_count|
+---------------+-------------------+----------+---------+----------------+--------------------+--------+---------------------+------------------+--------------------+---------------+
|      100005773|           20221228|De 34 años|MASCULINO|  S - SUBSIDIADO|100 - ACCIDENTE O...|   23807|                   SI|         TIERRALTA|        1 - VIVO (A)|              1|
|      100006019|           20220506|De 28 años| FEMENINO|C - CONTRIBUTIVO|110 - BAJO PESO A...|   11001|                   NO|       Bogotá D.C.|        1 - VIVO (A)|              1|
|      100009745|           19000101|De 27 años| FEMENINO|  S - SUBSIDIADO|110 -

+---------------+-------------------+----------+---------+----------------+--------------------+--------+---------------------+------------------+--------------------+---------------+
|PersonaBasicaID|FechaInicioSINTOmas|      Edad|     Sexo|     TipoRegimen|              Evento|    Area|PacienteHospitalizado|    CondicionFinal|DiagnosticoDefuncion|personaid_count|
+---------------+-------------------+----------+---------+----------------+--------------------+--------+---------------------+------------------+--------------------+---------------+
|      100005773|           20221228|De 34 años|MASCULINO|  S - SUBSIDIADO|100 - ACCIDENTE O...|   23807|                   SI|         TIERRALTA|        1 - VIVO (A)|              1|
|      100006019|           20220506|De 28 años| FEMENINO|C - CONTRIBUTIVO|110 - BAJO PESO A...|   11001|                   NO|       Bogotá D.C.|        1 - VIVO (A)|              1|
|      100009745|           19000101|De 27 años| FEMENINO|  S - SUBSIDIADO|110 -

In [25]:
# Contar el número de registros - me da satisfactoriamente la cantidad de registros en cada año ( 97.416)
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 30988


Varia un poco el numero de registros agrupados 30988 vs 30660 

In [26]:
 df_final =grouped_df


In [27]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 30988


In [28]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------------+-------------------+----------+---------+----------------+--------------------+--------+---------------------+------------------+--------------------+---------------+
|PersonaBasicaID|FechaInicioSINTOmas|      Edad|     Sexo|     TipoRegimen|              Evento|    Area|PacienteHospitalizado|    CondicionFinal|DiagnosticoDefuncion|personaid_count|
+---------------+-------------------+----------+---------+----------------+--------------------+--------+---------------------+------------------+--------------------+---------------+
|      100005773|           20221228|De 34 años|MASCULINO|  S - SUBSIDIADO|100 - ACCIDENTE O...|   23807|                   SI|         TIERRALTA|        1 - VIVO (A)|              1|
|      100006019|           20220506|De 28 años| FEMENINO|C - CONTRIBUTIVO|110 - BAJO PESO A...|   11001|                   NO|       Bogotá D.C.|        1 - VIVO (A)|              1|
|      100009745|           19000101|De 27 años| FEMENINO|  S - SUBSIDIADO|110 -

Se notifico que esta muestra tenia errores desde la fase de exploración, no se corrigieron... 

# Empieza el proceso de generación de tablas SIVIGILA

# Pagina 1 - Power BI

In [30]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

## 1. Agrupado por año, mes, Edad, Sexo y TipoRegimen

In [31]:
# Selecciona la columna 'FechaRegistro' y muestra sus datos
#df2.select('FechaInicioSINTOmas').show(100, truncate=False)

+-------------------+
|FechaInicioSINTOmas|
+-------------------+
|20221223           |
|20220228           |
|20221228           |
|20220506           |
|20220706           |
|19000101           |
|19000101           |
|19000101           |
|20221105           |
|19000101           |
|20220505           |
|19000101           |
|19000101           |
|19000101           |
|20220809           |
|20220921           |
|19000101           |
|20220928           |
|19000101           |
|19000101           |
|19000101           |
|19000101           |
|20221231           |
|20220731           |
|19000101           |
|19000101           |
|20221216           |
|19000101           |
|20220610           |
|20220820           |
|19000101           |
|19000101           |
|20220530           |
|19000101           |
|19000101           |
|19000101           |
|20221010           |
|20220613           |
|20220110           |
|20220727           |
|PRIMERA VEZ        |
|19000101           |
|20221030 

+----+----+-----------+----------------+------------------+--------------------+
| Año| Mes|       Sexo|     TipoRegimen|Número_de_Personas|Número_de_Atenciones|
+----+----+-----------+----------------+------------------+--------------------+
|null|null|   FEMENINO| N - NO AFILIADO|               115|                 115|
|null|null|De 06 meses|           MESES|                 1|                   1|
|null|null| De 38 años|            AÑOS|                 1|                   1|
|null|null|   FEMENINO|  S - SUBSIDIADO|             13506|               13554|
|null|null|  MASCULINO| N - NO AFILIADO|               164|                 166|
|null|null| De 21 años|            AÑOS|                 1|                   1|
|null|null| De 27 años|            AÑOS|                 2|                   2|
|null|null|      MESES|       MASCULINO|                 4|                   5|
|null|null|       AÑOS|       MASCULINO|                14|                  14|
|null|null|   FEMENINO|   E 

In [33]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, sexo y tipo de régimen
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'TipoRegimen').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+----------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|     TipoRegimen|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+----------------+------------------+--------------------+
|2022|  5| De 16 años| FEMENINO|  S - SUBSIDIADO|                17|                  17|
|2022|  8| De 26 años| FEMENINO|C - CONTRIBUTIVO|                18|                  18|
|2022|  5| De 27 años|MASCULINO| N - NO AFILIADO|                 1|                   1|
|2022|  1|De 06 meses| FEMENINO|  S - SUBSIDIADO|                 6|                   7|
|2022|  9| De 25 días| FEMENINO|  S - SUBSIDIADO|                 1|                   1|
|2022| 11| De 43 años| FEMENINO|  S - SUBSIDIADO|                 2|                   2|
|2022|  3| De 68 años|MASCULINO|  S - SUBSIDIADO|                 2|                   3|
|2022|  5| De 46 años| FEMENINO|  S - SUBSIDIADO|                 1|                   1|
|2022|  8|

Se compara los datos con el power BI dando satisfactorio los resultados
Se valida correctamente: 

Año	Mes	Edad	Sexo	TipoRegimen	# personas	# Atenciones
2022	enero	De 06 meses	FEMENINO	S - SUBSIDIADO	6	7
Año	Mes	Edad	Sexo	TipoRegimen	# personas	# Atenciones
2022	enero	De 19 años	MASCULINO	P - ESPECIAL	1	1

# 2. Agrupado por año, mes, Edad, Sexo y Evento

In [36]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y Evento
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'Evento').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+---------+--------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|              Evento|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+--------------------+------------------+--------------------+
|2022|  1| De 14 años|MASCULINO|100 - ACCIDENTE O...|                 3|                   3|
|2022|  1| De 04 años|MASCULINO|113 - DESNUTRICIÓ...|                16|                  16|
|2022| 12| De 32 años| FEMENINO|110 - BAJO PESO A...|                23|                  23|
|2022|  9| De 58 años|MASCULINO|100 - ACCIDENTE O...|                 4|                   4|
|2022|  3| De 66 años| FEMENINO|100 - ACCIDENTE O...|                 1|                   1|
|2022|  2| De 22 años| FEMENINO|110 - BAJO PESO A...|                31|                  31|
|2022|  2| De 24 años| FEMENINO|100 - ACCIDENTE O...|                 3|                   3|
|2022| 12| De 39 años| FEMENINO|100 - ACCIDENTE O...|       

Se compara los datos con el power BI dando satisfactorio los resultados
Se valida correctamente: 

Año	Mes	Edad	Sexo	Evento	# personas	# Atenciones
2022	enero	De 04 años	MASCULINO	113 - DESNUTRICIÓN AGUDA EN MENORES DE 5 AÑOS	16	16 

Año	Mes	Edad	Sexo	Evento	# personas	# Atenciones
2022	enero	De 48 años	MASCULINO	100 - ACCIDENTE OFIDICO	4	4

# 3.  Agrupado por año, mes, Edad, Sexo y Area

In [37]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y Area
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'Area').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+---------+-------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|               Area|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+-------------------+------------------+--------------------+
|2022|  9| De 28 años| FEMENINO|              68001|                 1|                   1|
|2022|  5| De 27 años| FEMENINO|              25200|                 1|                   1|
|2022| 10| De 28 años| FEMENINO|              76001|                 3|                   3|
|2022|  9| De 26 años| FEMENINO|              05360|                 1|                   1|
|2022|  2| De 29 años| FEMENINO|              25430|                 2|                   2|
|2022|  7| De 36 años| FEMENINO|              25290|                 1|                   1|
|2022|  7| De 12 años|MASCULINO|              23001|                 1|                   1|
|2022|  9| De 23 años| FEMENINO|              19548|                 1

# 4.  Agrupado por año, mes, Edad, Sexo y PacienteHospitalizado

In [38]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y PacienteHospitalizado
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'PacienteHospitalizado').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+---------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|PacienteHospitalizado|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+---------------------+------------------+--------------------+
|2022| 10| De 59 años| FEMENINO|                   SI|                 1|                   1|
|2022|  5| De 12 años|MASCULINO|                   SI|                 1|                   1|
|2022|  9| De 10 años| FEMENINO|                   NO|                 1|                   1|
|2022|  4| De 12 años| FEMENINO|                   SI|                 1|                   1|
|2022|  2| De 64 años| FEMENINO|                   NO|                 1|                   1|
|2022| 12| De 33 años| FEMENINO|                   NO|                 8|                   8|
|2022|  3| De 43 años|MASCULINO|                   SI|                 1|                   1|
|2022| 10| De 16 años|MASCULINO|                  

# 5.  Agrupado por año, mes, Edad, Sexo y CondicionFinal

In [39]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y CondicionFinal
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'CondicionFinal').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+--------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|      CondicionFinal|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+--------------------+------------------+--------------------+
|2022| 11| De 28 años| FEMENINO|               BELLO|                 1|                   1|
|2022|  3| De 27 años| FEMENINO|EL CARMEN DE VIBORAL|                 1|                   1|
|2022|  5| De 15 años| FEMENINO|             NECOCLÍ|                 1|                   1|
|2022| 11| De 24 años| FEMENINO|             JAMUNDÍ|                 1|                   1|
|2022|  6| De 25 años|MASCULINO|         SABANALARGA|                 1|                   1|
|2022|  4| De 04 años| FEMENINO|         Bogotá D.C.|                 3|                   3|
|2022|  6| De 21 años| FEMENINO|         SANTA MARTA|                 1|                   1|
|2022|  2| De 02 años| FEMENINO|         BUCARAMANGA|       

# 6.  Agrupado por año, mes, Edad, Sexo y DiagnosticoDefuncion

In [42]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaInicioSINTOmas', 'Edad', 'Sexo', 'TipoRegimen',
    'Evento', 'Area', 'PacienteHospitalizado', 'CondicionFinal', 'DiagnosticoDefuncion', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaInicioSINTOmas' a tipo fecha con el formato correcto, manejando errores ( por que vienen datos errones entre columnas)
df_filtered = df_filtered.withColumn(
    'FechaInicioSINTOmas',
    F.when(F.col('FechaInicioSINTOmas').rlike('^\d{8}$'), F.to_date('FechaInicioSINTOmas', 'yyyyMMdd'))
     .otherwise(None)  # Asignar None (nulo) a valores que no coinciden con el patrón de fecha
)

# Filtrar valores atípicos (por ejemplo, 19000101)
df_filtered = df_filtered.filter(
    (F.col('FechaInicioSINTOmas').isNotNull()) & (F.col('FechaInicioSINTOmas') > '1900-01-01')
)

# Extraer año y mes de la columna 'FechaInicioSINTOmas'
df_filtered = df_filtered.withColumn('Año', F.year('FechaInicioSINTOmas'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaInicioSINTOmas'))

# Agrupar por año, mes, Edad ,sexo y DiagnosticoDefuncion
result_df = df_filtered.groupBy('Año', 'Mes', 'Edad', 'Sexo', 'DiagnosticoDefuncion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+--------------------+------------------+--------------------+
| Año|Mes|       Edad|     Sexo|DiagnosticoDefuncion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+--------------------+------------------+--------------------+
|2022|  2| De 78 años|MASCULINO|        1 - VIVO (A)|                 1|                   1|
|2022|  7|De 06 meses| FEMENINO|   A419 - SEPTICEMIA|                 1|                   1|
|2022|  2|De 04 meses|MASCULINO|E43X - DESNUTRICI...|                 1|                   1|
|2022|  6| De 54 años|MASCULINO|        1 - VIVO (A)|                 3|                   3|
|2022|  8| De 77 años|MASCULINO|        1 - VIVO (A)|                 2|                   2|
|2022| 11| De 02 años| FEMENINO|        1 - VIVO (A)|                28|                  28|
|2022|  6|  De 01 año|MASCULINO|     J189 - NEUMONIA|                 1|                   1|
|2022|  3| De 38 años|MASCULINO|        1 - VIVO (A)|       

In [43]:
spark.stop()